### Exercicio 1

Neste exercício pretende-se criar uma class em Python que implemente o algoritmo de Boneh & Venkatesan (BV)

Nesta primeira classe foi definido o HiddenNumberProblem. A classe quando inicializada gera um segredo e contém funções que permitem calcular o msb e realizar a comparação do segredo gerado e do segredo calculado.

In [1]:
class HiddenNumber:
    
    def __init__(self,p):
        #gerar o segredo
        self.secret = ZZ.random_element(p)
        print 'The Secret is: '; print self.secret
    
    '''
    Aproximação dos k bits mais significativos do seu 
    argumento módulo p
    '''
    def msb(self,k,p,xi):
        val = ZZ(Mod(xi*self.secret,p))
        bits = val.digits(2)
        bits.reverse()
        final = ''
        if len(bits) < k:
            final = ''.join(str(e) for e in bits)
        else :
            for i in range(0,k):
                final += str(bits[i])
        return final

    def compare(self,calc):
        if calc == self.secret:
            return True
        else:
            return False
    


Nestas duas funções é gerado o vetor x e o vetor u.

O vetor x é constituido por l inteiros positivos gerados aleatoriamente no intervalo {0...p-1}

O vetor u é constituido por l inteiros positivos que verificam a igualdade ui = msb(s*xi)

In [2]:
def generate_xvector(l,p):
    x = []
    for i in range(0,l):
        x.append(ZZ.random_element(p))
    return x

def generate_uvector(xv,k,p,hnp):
    u = []
    for xi in xv:
        ui = hnp.msb(k,p,xi)
        #binario para decimal
        ui = ZZ(int(ui,2))
        u.append(ui)
    return u
    


### Algoritmo Boneh && Venkatesan

Nesta parte seguimos o link fornecido no enunciado.

Na inicialização da classe geramos o lambda, a matriz L, o target t e por fim o reticulado.

Por fim, resolve-se o CVP aproximado do reticulado com o target t (seguindo o exemplo na worksheet do TP4 HardLattice)

In [11]:
import sage.modules.free_module_integer as fmi
import numpy as np

class BV:
    
    def __init__(self,uv,xv,p,k,l):
        
        #gerar lambda, matriz L, target T e reticulado
        self.comp_lambda = 2^(k+1)
        self.L = self.comp_lambda * p * matrix.identity(l)
        self.L = self.L.insert_row(l,zero_vector(l))
        self.L = self.L.transpose()
        x = [self.comp_lambda * i for i in xv]
        x.append(1)
        self.L = self.L.insert_row(l,x)
        
        #gerar matriz target
        self.target = [self.comp_lambda * i for i in uv]
        self.target.append(0)
        self.target = matrix(self.target)
        self.retic = fmi.IntegerLattice(self.L)
    
    #CVP aproximado
    def CVP_resolve(self,x,p,l):
        L = matrix(self.retic.reduced_basis)
        t = matrix(1,l+1,list(-self.target))
        zero = matrix(l+1,1,[0]*(l+1))
        M = matrix(1,1,[p**2])
        L1 = block_matrix(2,2,[[L,zero],[t,M]])
        #base reduzida do reticulado de base L1
        Lred = fmi.IntegerLattice(L1).reduced_basis
        err1 = np.array(Lred[l+1][:-1])
        y1 = err1 + self.target
        # última componente do vetor resultante
        return y1[0][l]
        

### Teste ao algoritmo de Boneh e Venkatesan

In [33]:
l = 2^4
k = 32
p = 2^32
xv = generate_xvector(l,p)
hnp = HiddenNumber(p)
uv = generate_uvector(xv,k,p,hnp)
bv = BV(uv,xv,p,k,l)
calc = bv.CVP_resolve(xv,p,l)
print('Calculated Secret is: '); print(calc)
if hnp.compare(calc):
    print 'True'
else:
    print 'False'

The Secret is: 
130685910
Calculated Secret is: 
130685910
True
